# Financial Econometrics I: Homework 2

## David Černý, Jan Hrušák

# Problem 1

In [ ]:
library(quantmod)
library(rugarch)
library(dplyr)


set.seed(38848152)

data = read.csv("symbols2.csv")

symbols = as.vector(sample(data$Symbol, 100, replace = FALSE))


DownloadYahoo <- function(symbol, from, to = Sys.Date())
{
  # Package Check
  if (!requireNamespace("quantmod", quietly = TRUE)) 
  {
    stop("Package 'quantmod' must be installed. Use install.packages('quantmod').")
  }
  
  # Download data from yahoo finance
  tryCatch(
  {
    getSymbols(symbol, from = from, to = to, auto.assign = TRUE)
    data <- get(symbol)
    names(data) = unname(sapply(names(data), sub, pattern = ".*\\.", replacement = ""))
    return(data)
  }, error = function(e) 
  {
    message("Skipping due to error: ", e$message)
    return(NULL)
  })
}


In [ ]:

from = as.Date('2020-01-01')
to = as.Date('2022-11-01')

dfs = list()
pricesList = list()

#names(prices) = symbols
for (i in 1 : length(symbols))
{
  
  df = DownloadYahoo(symbols[i], from, to)
  # Skip to the next iteration if df is NULL (= unsuccessful import)
  if (is.null(df)) next
  # Assign using double square brackets for list
  dfs[[i]] = df 
}
names(dfs) = symbols

# Create new List without the invalid symbols
for (symbol in symbols)
{
  if(!is.null(dfs[[symbol]]))
  {
    pricesList[[symbol]] = dfs[[symbol]]
  }
}
length(pricesList)
prices = data.frame("Date" = index(pricesList[[1]]))
# Create Data Frame of prices
for (validSymbol in names(pricesList))
{
  prices[validSymbol] = as.vector(pricesList[[validSymbol]]$Adjusted)
}


In [ ]:

### 1)
# Compute Log Returns
logReturns = data.frame("Date" = prices$Date[2:length(prices$Date)])
for (validSymbol in names(pricesList))
{
  logReturns[validSymbol] = diff(log(as.vector(pricesList[[validSymbol]]$Adjusted)))
}
summary(logReturns)
# Delete the column with almost 500 NAs
logReturns = select(logReturns, -COL)
sum(is.na(logReturns))
validSymbols = colnames(logReturns)[2:ncol(logReturns)]


We computed the logarithmic returns of each symbol in our collection, then we
omitted all NAs in the new dataframe, since otherwise it would not be 
compatible with the following processes. Due to this data cleaning we removed no row, since all NAs was included in the column of symbol "COL". In case of just a few NAs it would be more suitable to delete all rows containing NAs, but since all of them are in one column, we should delete it.


In [ ]:
library(rugarch)
### 2)
# For each stock, estimate the parameters of a GARCH(1, 1) model:
garchModels = list()

for (validSymbol in validSymbols)
{
  spec = ugarchspec(variance.model = list(model = "sGARCH", garchOrder = c(1, 1)), 
                    mean.model = list(armaOrder = c(0, 0), include.mean = TRUE), 
                    distribution.model = "norm")
  garchModels[[validSymbol]] = ugarchfit(spec, logReturns[, validSymbol])
}


We estimated the GARCH(1,1) model for each symbol of our dataframe using the 
"rugarch" package.


In [ ]:

### 3)
# Plotting Garch(1,1) coefs
alphas = sapply(garchModels, function(model) coef(model)["alpha1"])
betas = sapply(garchModels, function(model) coef(model)["beta1"])
alphasPlusBetas = alphas + betas

par(mfrow = c(1, 3))
hist(alphas, main = "Histogram of alphas", xlab = "Alpha")
hist(betas, main = "Histogram of betas", xlab = "Beta")
hist(alphasPlusBetas, main = "Histogram of alphas + betas", xlab = "Alpha + Beta")


$\alpha_i$ represents the impact of the squared residuals (innovations) on the 
conditional variance. A higher value of $\alpha_i$ indicates that the volatility
is more sensitive to recent market shocks.

$\beta_i$ represents the persistence of the conditional variance (the impact of past
volatility on current volatility). A higher value of $\beta_i$ indicates that the
volatility is more persistent and takes longer to dissipate.

The sum $\alpha_i$ + $\beta_i$ measures the rate at which the response to volatility decays
over time. A value close to 1 indicates a slower decay and more persistence
in volatility.

In [ ]:

### 4)
# Find the minimum and maximum values of alpha_i, beta_i, and alpha_i + beta_i. Comment briefly:
min_alpha = min(alphas)
max_alpha = max(alphas)
min_beta = min(betas)
max_beta = max(betas)
min_alphaPlusBeta = min(alphasPlusBetas)
max_alphaPlusBeta = max(alphasPlusBetas)

cat("Minimum alpha:", min_alpha, "\n")


The value is very close to zero (practically zero).
This indicates that for the stock with the lowest alpha, the impact of recent
market shocks on volatility is very minimal.

In [ ]:
cat("Maximum alpha:", max_alpha, "\n")

The maximal alpha is relatively high.
This suggests that for the stock with the highest alpha, the volatility is highly sensitive to recent market shocks, and new information or innovations have a significant impact on the conditional variance.

In [ ]:
cat("Minimum beta:", min_beta, "\n")


The minimal value of beta is relativelly low but still positive.
This implies that for the stock with the lowest beta, the persistence of volatility is relatively low, and past volatility has a smaller impact on current volatility.

In [ ]:
cat("Maximum beta:", max_beta, "\n")


The maximal beta is almost 1.
This indicates that for the stock with the highest beta, the volatility is highly persistent, and past volatility has a significant impact on current volatility. The conditional variance is heavily influenced by its own lagged values.

In [ ]:
cat("Minimum alpha + beta:", min_alphaPlusBeta, "\n")


The minimal value of the sum is relatively low.
This suggests that for the stock with the lowest alpha + beta, the response to volatility decays relatively quickly over time, and the impact of past shocks on future volatility diminishes faster.

In [ ]:
cat("Maximum alpha + beta:", max_alphaPlusBeta, "\n")


Similarly to maximal beta, the maximal value of the sum is also almost 1.
This indicates that for the stock with the highest alpha + beta, the response to volatility decays slowly over time, and the impact of past shocks on future volatility persists for a longer period. The conditional variance exhibits strong persistence.

In [ ]:

### 5)
marketVolatility = data.frame("Date" = logReturns$Date)

for (i in 1:nrow(logReturns))
{
  dayVolatilities = sapply(garchModels, function(model) if (!is.null(model)) sigma(model)[i] else NA)
  marketVolatility$Median[i] = median(dayVolatilities, na.rm = TRUE)
  marketVolatility$Quantile95[i] = quantile(dayVolatilities, 0.95, na.rm = TRUE)
  marketVolatility$Quantile05[i] = quantile(dayVolatilities, 0.05, na.rm = TRUE)
}

plot(marketVolatility$Date, marketVolatility$Median, type = "l", xlab = "Date", ylab = "Market Volatility",
     main = "Market Volatility and Quantiles", ylim = range(marketVolatility[, 2:4], na.rm = TRUE))
lines(marketVolatility$Date, marketVolatility$Quantile95, col = "blue")
lines(marketVolatility$Date, marketVolatility$Quantile05, col = "red")
legend("topright", legend = c("Median", "95% Quantile", "5% Quantile"),
       col = c("black", "blue", "red"), lty = 1)


We plotted the estimated market volatility and its "low" and "high" quantiles.
In the figure we can see that the volatility appears to be most of the time 
quite stable, although, there are also visible at least two high volatility 
clusters - we might assume that the first cluster (the most significant one) 
is likely to be impacted by the covid19 crisis hit. The explanation for the 
second one does not seem to be apparent..

# Problem 2

## Get the data:

We get the data again, since we used also our second seed number.

In [ ]:
library(quantmod)

# seed
set.seed(53600945)

# generate 100 random integers from the range 1 to 377
random_indices <- sample(1:377, 100, replace = FALSE)

# read tickers
symbols_df <- read.csv("symbols2.csv", sep = ",", header = TRUE)

#print(symbols_df)

#print(random_indices)


# select tickers based on the generated random numbers
selected_tickers <- symbols_df$Symbol[random_indices]


print(selected_tickers)

# start and end dates for the data download
start_date <- as.Date("2020-01-01")
end_date <- as.Date("2022-10-31")

# list to store the stock data
stock_data_list <- list()

# loop through the selected tickers and download the stock price data
for(ticker in selected_tickers) {
  # tryCatch fction is used to ignore errors (if the stock is not available etc.)
  stock_data <- tryCatch({
    getSymbols(ticker, src = 'yahoo', from = start_date, to = end_date, auto.assign = FALSE)
  }, error = function(e) NULL)
  
  # if stock data was downloaded, add it to the list
  if (!is.null(stock_data)) {
    stock_data_list[[ticker]] <- stock_data
  }
}

#print(head(stock_data_list))

## Compute the log returns:

In [ ]:
# fction to compute logarithmic returns
compute_log_returns <- function(stock_prices) {
  
  log_returns <- diff(log(Cl(stock_prices)))
  return(log_returns)
}

# a list to store the logarithmic returns for each stock
log_returns_list <- list()

# loop through each of the stock's data in 'stock_data_list' to compute the logarithmic returns
for(ticker in names(stock_data_list)) {
  stock_prices <- stock_data_list[[ticker]]
  log_returns <- compute_log_returns(stock_prices)
  log_returns_list[[ticker]] <- log_returns
}

#print(log_returns_list)


## Compute mean log returns:

In [ ]:

# combine all the logarithmic returns into one dataframe
# each column represents a stock, and each row represents a date
combined_log_returns <- do.call(merge, c(log_returns_list, all = TRUE))

# mean logarithmic return for each date
mean_log_returns <- rowMeans(combined_log_returns, na.rm = TRUE)

# convert the mean logarithmic returns to a dataframe 
mean_log_returns_df <- data.frame(Date = index(combined_log_returns), MeanLogReturn = mean_log_returns)

head(mean_log_returns_df)


# ARMA identification and estimation:

In [ ]:
# clean the data from missing observations
clean_data <- na.omit(mean_log_returns_df)


In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

plot.ts(clean_data$MeanLogReturn, ylab = NA, main = 'mean_log_returns')

 We can see a pattern typical for financial returns data, that are very likely mean stationary. There seem to be clusters of higher volatility, with the most significant one at the beginning of our data, which covers mainly the year 2020. We might argue that this was due to the onset of COVID-19 and related economic uncertainty. The mean stationarity of the series is further tested using ADF and KPSS tests below.

In [ ]:
#install.packages('tseries')

rets = clean_data$MeanLogReturn

In [ ]:
library(tseries)


Box.test(rets, type = 'Ljung-Box')

adf.test(rets, k = 1)

kpss.test(rets, null = 'Level')
kpss.test(rets, null = 'Trend')

The Ljung-Box Q test rejects the null hypothesis of no autocorrelation. This suggests that past values might have a predictive power on future values in our series.

Based on the ADF test, we reject the null hypothesis of unit root; suggesting mean stationarity. Since ADF test tests just for a specific form of non-stationarity (unit root) we conduct a KPSS test. 

Based on the KPSS test, we do not reject the null hypothesis of neither level (around a mean) or trend (around a trend of the series) stationarity. All on the 5% sign. level.

In [ ]:

options(repr.plot.width = 12, repr.plot.height = 6)
par(mfrow = c(1, 2))


# autocorrelation and partial autocorrelation fctions
acf(rets)
pacf(rets)

#print(clean_data)

Based on the plotted PACF we estimate AR(2), AR(4), and AR(7). We chose those based on the displayed significant lags. 

We suspect that AR(9) might work the best, however it is also a model of a probably too high order, possibly leading to overfitting our data.

Since it is not clear from the plots whether to use AR or MA model, we later estimate MA models and the combination of both model types as well. For now we focus on the AR models only.

In [ ]:
library(forecast)
rets_ar2 <- Arima(rets, order = c(2, 0, 0))
summary (rets_ar2)

par(mfrow = c(1, 2))
acf(rets_ar2$residuals, main = NA)
pacf(rets_ar2$residuals, main = NA)

In [ ]:
Box.test(rets_ar2$residuals, type = "Ljung-Box", lag = 1)

Box.test(rets_ar2$residuals, type = "Ljung-Box", lag = 4)

Box.test(rets_ar2$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets_ar2$residuals, type = "Ljung-Box", lag = 12)

From the PACF of the estimated AR(2) model we can see that it does not 'smoothen' the autocorellation well. This can be seen also on the low p-values of the Box-Ljung tests at the higher lags. Hence, there is still a room for a better model.

In [ ]:
library(forecast)
rets_ar4 <- Arima(rets, order = c(4, 0, 0))
summary (rets_ar4)

par(mfrow = c(1, 2))
acf(rets_ar4$residuals, main = NA)
pacf(rets_ar4$residuals, main = NA)

Box.test(rets_ar4$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets_ar4$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets_ar4$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets_ar4$residuals, type = "Ljung-Box", lag = 12)

Similarly as for the AR(2) model, the AR(4) does not have a very good fit based on both PACF and the B-L tests.

In [ ]:
library(forecast)
rets_ar7 <- Arima(rets, order = c(7, 0, 0))
summary (rets_ar7)

par(mfrow = c(1, 2))
acf(rets_ar7$residuals, main = NA)
pacf(rets_ar7$residuals, main = NA)

Box.test(rets_ar7$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets_ar7$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets_ar7$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets_ar7$residuals, type = "Ljung-Box", lag = 12)

The AR(7) model seems to have much better fit based on all the p-values above the 5% sign. threshold of the B-L tests for various lags. However if one looks at the PACF there are still two significant lags.

In [ ]:
library(forecast)
rets_ar9 <- Arima(rets, order = c(9, 0, 0))
summary (rets_ar9)

par(mfrow = c(1, 2))
acf(rets_ar9$residuals, main = NA)
pacf(rets_ar9$residuals, main = NA)

Box.test(rets_ar9$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets_ar9$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets_ar9$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets_ar9$residuals, type = "Ljung-Box", lag = 12)

As expected the AR(9) works quite well with all the B-L tests p-values almost 1 and one barely significant lag on the PACF. However we continue with model identification since we want as parsimonious model as possible.

Based on the ACF of our returns data plotted earlier we suspect an MA(1) process:

In [ ]:
library(forecast)
rets_ma1 <- Arima(rets, order = c(0, 0, 1))
summary (rets_ma1)

par(mfrow = c(1, 2))
acf(rets_ma1$residuals, main = NA)
pacf(rets_ma1$residuals, main = NA)

Box.test(rets_ma1$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets_ma1$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets_ma1$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets_ma1$residuals, type = "Ljung-Box", lag = 12)

This model does not look very well based on both the B-L tests for higher lags as well as the plot. 

We try the auto-arima fction:

In [ ]:
auto.arima(rets)

Thus we try the suggested MA(2) model:

In [ ]:
library(forecast)
rets_ma2 <- Arima(rets, order = c(0, 0, 2), include.mean = FALSE)
summary (rets_ma2)

par(mfrow = c(1, 2))
acf(rets_ma2$residuals, main = NA)
pacf(rets_ma2$residuals, main = NA)


Box.test(rets_ma2$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets_ma2$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets_ma2$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets_ma2$residuals, type = "Ljung-Box", lag = 12)

This model, however, behaves not very differently from the MA(1) model. It does not have much better fit.

Therefore, we try a combination of the AR and MA models:

In [ ]:
library(forecast)
rets_arma42 <- Arima(rets, order = c(4, 0, 2))
summary (rets_arma42)

par(mfrow = c(1, 2))
acf(rets_arma42$residuals, main = NA)
pacf(rets_arma42$residuals, main = NA)

Box.test(rets_arma42$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets_arma42$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets_arma42$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets_arma42$residuals, type = "Ljung-Box", lag = 12)

The ARMA(4,2) model seems to be the best model based on the ACF, PACF, as well as the tests (all p-values much higher than 0.05). ACF as well as PACF have no significant lags.

Next we compare the estimated models based on the AIC and BIC criteria:

In [ ]:
models <- 7
criteria <- matrix(ncol = 2, nrow = models)
colnames(criteria) <- c('AIC', 'BIC')
rownames(criteria) <- c('AR(2)','AR(4)', 'AR(7)', 'AR(9)','ARMA(4,2)','MA(1)','MA(2)')

criteria[1, 1] <- rets_ar2$aic
criteria[1, 2] <- rets_ar2$bic
criteria[2, 1] <- rets_ar4$aic
criteria[2, 2] <- rets_ar4$bic
criteria[3, 1] <- rets_ar7$aic
criteria[3, 2] <- rets_ar7$bic

criteria[4, 1] <- rets_ar9$aic
criteria[4, 2] <- rets_ar9$bic
criteria[5, 1] <- rets_arma42$aic
criteria[5, 2] <- rets_arma42$bic
criteria[6, 1] <- rets_ma1$aic
criteria[6, 2] <- rets_ma1$bic
criteria[7, 1] <- rets_ma2$aic
criteria[7, 2] <- rets_ma2$bic

criteria

We would choose ARMA(4,2) due to the lowest values apart from the AIC of the AR(9) model.

The final plot of fitted vs actual follows:

In [ ]:
plot.ts(rets, ylab = NA, main = 'AR(2) fitted values')
lines(rets_ar2$fitted, col = 'red')

plot.ts(rets, ylab = NA, main = 'AR(9) fitted values')
lines(rets_ar9$fitted, col = 'red')

plot.ts(rets, ylab = NA, main = 'MA(2) fitted values')
lines(rets_ma2$fitted, col = 'red')

plot.ts(rets, ylab = NA, main = 'ARMA(4,2) fitted values')
lines(rets_arma42$fitted, col = 'red')

In [ ]:
# AR(2) Model
plot.ts(rets[1:100], ylab = NA, main = 'AR(2) fitted values for first 100 observations')
lines(rets_ar2$fitted[1:100], col = 'red')

# AR(9) Model
plot.ts(rets[1:100], ylab = NA, main = 'AR(9) fitted values for first 100 observations')
lines(rets_ar9$fitted[1:100], col = 'red')

# MA(2) Model
plot.ts(rets[1:100], ylab = NA, main = 'MA(2) fitted values for first 100 observations')
lines(rets_ma2$fitted[1:100], col = 'red')

# ARMA(4,2) Model
plot.ts(rets[1:100], ylab = NA, main = 'ARMA(4,2) fitted values for first 100 observations')
lines(rets_arma42$fitted[1:100], col = 'red')

## Divide the data to two parts - since the models are fitting differently for the higher volatility period than the rest:

Based on the actual vs fitted values graphs we can see that in the cluster of the high volatility, the models seem to fit the data well. However, in the rest, the fit seems to be worse. Therefore we would divide the data to two parts and identify the models for both parts again...

In [ ]:
rets1 <- rets[1:(length(rets) / 2)]
rets2 <- rets[((length(rets) / 2) + 1):length(rets)]

options(repr.plot.width = 10, repr.plot.height = 10)
par(mfrow = c(2, 2))

acf(rets1, main = "First subsample")
pacf(rets1, main = "First subsample")
acf(rets2, main = "Second subsample")
pacf(rets2, main = "Second subsample")

In [ ]:
Box.test(rets1, type = 'Ljung-Box')

adf.test(rets1, k = 1)

kpss.test(rets1, null = 'Level')
kpss.test(rets1, null = 'Trend')

In [ ]:
Box.test(rets2, type = 'Ljung-Box')

adf.test(rets2, k = 1)

kpss.test(rets2, null = 'Level')
kpss.test(rets2, null = 'Trend')

Based on the B-L test on the second half of the data it is likely that there is not enough autocorrelation to be able to model the data...

On the other hand the first part of the data seems to be appropriate for modelling.

For the first subsample we might expect similar models and results as for the whole, undivided dataset, since it seems the previous results were driven mainly by the variation in the first half of our dataset.

The analysis just of the first part of the divided dataset follows. Nevertheless, it is almost identical to the analysis of the whole dataset conducted above. Thus the following analysis is just illustrative, without any comments.

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

# ar2
library(forecast)
rets1_ar2 <- Arima(rets1, order = c(2, 0, 0))
summary (rets1_ar2)

par(mfrow = c(1, 2))
acf(rets1_ar2$residuals, main = NA)
pacf(rets1_ar2$residuals, main = NA)


Box.test(rets1_ar2$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets1_ar2$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets1_ar2$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets1_ar2$residuals, type = "Ljung-Box", lag = 12)


# ar4
library(forecast)
rets1_ar4 <- Arima(rets1, order = c(4, 0, 0))
summary (rets1_ar4)

par(mfrow = c(1, 2))
acf(rets1_ar4$residuals, main = NA)
pacf(rets1_ar4$residuals, main = NA)

Box.test(rets1_ar4$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets1_ar4$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets1_ar4$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets1_ar4$residuals, type = "Ljung-Box", lag = 12)



# ar7
library(forecast)
rets1_ar7 <- Arima(rets1, order = c(7, 0, 0))
summary (rets1_ar7)

par(mfrow = c(1, 2))
acf(rets1_ar7$residuals, main = NA)
pacf(rets1_ar7$residuals, main = NA)

Box.test(rets1_ar7$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets1_ar7$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets1_ar7$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets1_ar7$residuals, type = "Ljung-Box", lag = 12)


# ar9
library(forecast)
rets1_ar9 <- Arima(rets1, order = c(9, 0, 0))
summary (rets1_ar9)

par(mfrow = c(1, 2))
acf(rets1_ar9$residuals, main = NA)
pacf(rets1_ar9$residuals, main = NA)

Box.test(rets1_ar9$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets1_ar9$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets1_ar9$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets1_ar9$residuals, type = "Ljung-Box", lag = 12)



# ma1
library(forecast)
rets1_ma1 <- Arima(rets1, order = c(0, 0, 1))
summary (rets1_ma1)

par(mfrow = c(1, 2))
acf(rets1_ma1$residuals, main = NA)
pacf(rets1_ma1$residuals, main = NA)

Box.test(rets1_ma1$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets1_ma1$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets1_ma1$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets1_ma1$residuals, type = "Ljung-Box", lag = 12)



In [ ]:
auto.arima(rets1)

In [ ]:
library(forecast)
rets1_ma2 <- Arima(rets1, order = c(0, 0, 2), include.mean = FALSE)
summary(rets1_ma2)

par(mfrow = c(1, 2))
acf(rets1_ma2$residuals, main = NA)
pacf(rets1_ma2$residuals, main = NA)


Box.test(rets1_ma2$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets1_ma2$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets1_ma2$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets1_ma2$residuals, type = "Ljung-Box", lag = 12)

In [ ]:
library(forecast)
rets1_arma42 <- Arima(rets1, order = c(4, 0, 2))
summary (rets1_arma42)

par(mfrow = c(1, 2))
acf(rets1_arma42$residuals, main = NA)
pacf(rets1_arma42$residuals, main = NA)

Box.test(rets1_arma42$residuals, type = "Ljung-Box", lag = 1)
Box.test(rets1_arma42$residuals, type = "Ljung-Box", lag = 4)
Box.test(rets1_arma42$residuals, type = "Ljung-Box", lag = 8)
Box.test(rets1_arma42$residuals, type = "Ljung-Box", lag = 12)

In [ ]:
models <- 7
criteria <- matrix(ncol = 2, nrow = models)
colnames(criteria) <- c('AIC', 'BIC')
rownames(criteria) <- c('AR(2)','AR(4)', 'AR(7)', 'AR(9)','ARMA(4,2)','MA(1)','MA(2)')

criteria[1, 1] <- rets1_ar2$aic
criteria[1, 2] <- rets1_ar2$bic
criteria[2, 1] <- rets1_ar4$aic
criteria[2, 2] <- rets1_ar4$bic
criteria[3, 1] <- rets1_ar7$aic
criteria[3, 2] <- rets1_ar7$bic

criteria[4, 1] <- rets1_ar9$aic
criteria[4, 2] <- rets1_ar9$bic
criteria[5, 1] <- rets1_arma42$aic
criteria[5, 2] <- rets1_arma42$bic
criteria[6, 1] <- rets1_ma1$aic
criteria[6, 2] <- rets1_ma1$bic
criteria[7, 1] <- rets1_ma2$aic
criteria[7, 2] <- rets1_ma2$bic

criteria

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

plot.ts(rets1, ylab = NA, main = 'AR(2) fitted values')
lines(rets1_ar2$fitted, col = 'red')

plot.ts(rets1, ylab = NA, main = 'AR(9) fitted values')
lines(rets1_ar9$fitted, col = 'red')

plot.ts(rets1, ylab = NA, main = 'MA(2) fitted values')
lines(rets1_ma2$fitted, col = 'red')

plot.ts(rets1, ylab = NA, main = 'ARMA(4,2) fitted values')
lines(rets1_arma42$fitted, col = 'red')

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 6)

# AR(2) Model
plot.ts(rets1[1:100], ylab = NA, main = 'AR(2) fitted values for first 100 observations')
lines(rets1_ar2$fitted[1:100], col = 'red')

# AR(9) Model
plot.ts(rets1[1:100], ylab = NA, main = 'AR(9) fitted values for first 100 observations')
lines(rets1_ar9$fitted[1:100], col = 'red')

# MA(2) Model
plot.ts(rets1[1:100], ylab = NA, main = 'MA(2) fitted values for first 100 observations')
lines(rets1_ma2$fitted[1:100], col = 'red')

# ARMA(4,2) Model
plot.ts(rets1[1:100], ylab = NA, main = 'ARMA(4,2) fitted values for first 100 observations')
lines(rets1_arma42$fitted[1:100], col = 'red')